Documentation de la base de données disponible ici : https://challengedata.ens.fr/challenges/87 
-> Pb potentiel : 96% valeurs manquantes pour gender_pay_gap


Prendre une mesure de pollution/capita 

Discrimination puisse se faire également par l’obtention du job ? Gender gap qui mesure de façon brut les différences entres hommes et femmes au sein de l’entreprise sans contrôler des différences de niveau de formation, d’âge

Regarder un agrégat des scopes intra sectoriel / comparer par scope entre secteurs / calculer la contribution de chaque scope dans la somme por chercher cohérence avec le secteur

Stats desc :

o	OCDE -> taux de féminisation
o	Eurostat -> taux de féminisation par secteur, par pays


In [1]:
!pip install chardet
!pip install openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.4/199.4 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 22.2 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import linregress
import chardet
import math
from statistics import mode

In [ ]:
pd.set_option("display.max_columns", 35)
pd.set_option("display.max_rows", 15)

In [7]:
# Chargement de toutes les données
file_raw_path = "data/ESG_raw_data_07_02.xlsx"
raw_data = pd.read_excel(file_raw_path)

In [12]:
# Variables disponibles, toutes numériques
variables = ["market_cap", "employees", "revenue", "scope_1", "scope_2", "scope_3",
                         "waste_production", "waste_recycling", "water_consumption",
                         "water_withdrawal", "energy_consumption", "hours_of_training",
                         "gender_pay_gap", "independent_board_members_percentage",
                         "legal_costs_paid_for_controversies", "ceo_compensation"]

variables_env = ["scope_1", "scope_2", "scope_3",
                "waste_production", "waste_recycling", "water_consumption",
                "water_withdrawal", "energy_consumption"]

variables_eg = ["hours_of_training","gender_pay_gap", "independent_board_members_percentage",
                "legal_costs_paid_for_controversies", "ceo_compensation"]

# Années 
annees = ["2018", "2019", "2020"]

# Regions 
regions = ["Africa / Middle East", "Asia / Pacific", "Europe", 
           "Latin America and Caribbean", "United States and Canada"]

# Secteurs 
secteurs = raw_data['primary_industry'].unique()


raw_data.head()

,company_id,year,company_name,ticker,LEI,isin,Business Desc.,region,hq_country,primary_industry,...,waste_production,waste_recycling,water_consumption,water_withdrawal,energy_consumption,hours_of_training,independent_board_members_percentage,legal_costs_paid_for_controversies,ceo_compensation,gender_pay_gap
0,c67ee84d-5cf4-4cd0-85b9-027ad082748f,2020,Apple Inc.,NasdaqGS:AAPL,HWUPKR0MPOU8FGXBT394,US0378331005,"Apple Inc. designs, manufactures, and markets ...",United States and Canada,United States,"Technology Hardware, Storage and Peripherals",...,45705.80000,28939.8200,4.875000e+06,4.875000e+06,3.019170e+06,NaN,0.008571,0.00,14769259.0,NaN
1,c67ee84d-5cf4-4cd0-85b9-027ad082748f,2019,Apple Inc.,NasdaqGS:AAPL,HWUPKR0MPOU8FGXBT394,US0378331005,"Apple Inc. designs, manufactures, and markets ...",United States and Canada,United States,"Technology Hardware, Storage and Peripherals",...,58395.94000,32806.4100,4.890152e+06,4.992424e+06,2.889680e+06,NaN,0.008571,1.85,11555466.0,NaN
2,c67ee84d-5cf4-4cd0-85b9-027ad082748f,2018,Apple Inc.,NasdaqGS:AAPL,HWUPKR0MPOU8FGXBT394,US0378331005,"Apple Inc. designs, manufactures, and markets ...",United States and Canada,United States,"Technology Hardware, Storage and Peripherals",...,52849.83673,30104.5941,4.765152e+06,4.924242e+06,2.676460e+06,NaN,0.008750,NaN,15682219.0,NaN
3,9ea5e71e-33ac-4916-9c58-50ce9fa54abe,2020,Saudi Arabian Oil Company,SASE:2222,5586006WD91QHB7J4X50,SA14TG012N13,Saudi Arabian Oil Company operates as an integ...,Africa / Middle East,Saudi Arabia,Integrated Oil and Gas,...,NaN,NaN,3.615000e+07,3.615000e+07,1.495640e+08,NaN,0.454540,0.00,NaN,NaN
4,9ea5e71e-33ac-4916-9c58-50ce9fa54abe,2019,Saudi Arabian Oil Company,SASE:2222,5586006WD91QHB7J4X50,SA14TG012N13,Saudi Arabian Oil Company operates as an integ...,Africa / Middle East,Saudi Arabia,Integrated Oil and Gas,...,NaN,NaN,NaN,NaN,1.212160e+08,NaN,0.454540,NaN,NaN,NaN


In [14]:
all_r2 = []
count = 0

#for x in variables_env, y in variables_eg, a in annees, r in regions, s in secteurs :
#On ne conserve que les secteurs avec + de 5 entreprises / + de 5 caractéristiques
# 6160 au total
for x in variables_env :
    for y in variables_eg :
        for a in ["2020"] :
            for r in ["Europe"] :
                for s in secteurs :
                    count+= 1
                    data = pd.DataFrame(raw_data[(raw_data["year"] == int(a)) & (raw_data["primary_industry"] == s) & (raw_data["region"] == r)])
                    if data[x].size > 5 and data[y].size > 5 :
                        slope, intercept, r_value, p_value, std_err = linregress(data[x], data[y])
                        r_squared = r_value ** 2
                        all_r2.append([r_squared, s])
print(count)

6160


In [17]:
#Stats rapides sur les données numériques
def stats(liste):
    m = mode(liste)
    if np.isnan(m):
        return([np.mean(liste),np.min(liste),np.max(liste),np.std(liste),np.mean(np.log1p(liste)),np.min(np.log1p(liste)),np.max(np.log1p(liste)),np.std(np.log1p(liste)),len(liste.unique()),m,'NaN'])
    return([np.mean(liste),np.min(liste),np.max(liste),np.std(liste),np.mean(np.log1p(liste)),np.min(np.log1p(liste)),np.max(np.log1p(liste)),np.std(np.log1p(liste)),len(liste.unique()),m,liste.value_counts()[m]])

#Toutes données
listestats=[]
for i in variables:
    listestats.append(stats(imp_data[i]))
datastats = pd.DataFrame(np.transpose(listestats),columns=variables,index=['moyenne','min','max','std','moyennelog','minlog','maxlog','stdlog','nb_values','mode','nb_mode'])
display(datastats)

#Par régions
for j in regions:
    listestats=[]
    datar=raw_data[imp_data['region']==j]
    for i in variables:
        listestats.append(stats(datar[i]))
    datastats = pd.DataFrame(np.transpose(listestats),columns=variables,index=['moyenne','min','max','std','moyennelog','minlog','maxlog','stdlog','nb_values','mode','nb_mode'])
    print(j)
    display(datastats)

/opt/mamba/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/mamba/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/mamba/lib/python3.11/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,market_cap,employees,revenue,scope_1,scope_2,scope_3,waste_production,waste_recycling,water_consumption,water_withdrawal,energy_consumption,hours_of_training,gender_pay_gap,independent_board_members_percentage,legal_costs_paid_for_controversies,ceo_compensation
moyenne,6.232363e+03,1.206209e+04,3368.859507,1.485126e+06,4.487959e+05,6.857569e+06,1.370462e+07,6.269067e+05,4.688583e+07,1.967612e+08,3.711305e+07,5.469885e+05,2.327466e+00,0.563926,58.326954,1.374218e+07
min,3.676000e+01,1.000000e+00,-13944.090000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-8.100000e+00,0.000000,0.000000,9.768000e+03
max,2.251352e+06,2.300000e+06,460983.530000,5.538870e+08,4.059380e+08,1.679000e+10,2.797400e+10,8.434910e+07,6.934110e+10,9.156512e+10,8.167006e+10,1.482950e+08,4.420000e+01,1.000000,14830.200000,5.929000e+09
std,3.977866e+04,3.909027e+04,12976.189513,7.718823e+06,3.674240e+06,1.248129e+08,1.598835e+08,1.481749e+06,6.435403e+08,1.292035e+09,4.196974e+08,1.766736e+06,2.295728e+00,0.178272,218.937707,4.097939e+07
moyennelog,7.369190e+00,8.178858e+00,6.464067,1.267621e+01,1.197117e+01,1.389494e+01,1.530179e+01,1.287832e+01,1.623662e+01,1.772220e+01,1.672006e+01,1.266955e+01,-inf,0.440671,2.672291,1.631756e+01
minlog,3.631250e+00,6.931472e-01,-0.430783,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-inf,0.000000,0.000000,9.186969e+00
maxlog,1.462704e+01,1.464842e+01,13.041120,2.013247e+01,1.982171e+01,2.354405e+01,2.405454e+01,1.825047e+01,2.496230e+01,2.524032e+01,2.512595e+01,1.881471e+01,3.811097e+00,0.693147,9.604488,2.250312e+01
stdlog,1.312375e+00,1.582770e+00,1.806243,2.923557e+00,2.207257e+00,3.059139e+00,2.504685e+00,1.473947e+00,2.302496e+00,2.580689e+00,1.941337e+00,1.537337e+00,NaN,0.114656,2.014958,4.612749e-01
nb_values,1.319800e+04,2.044100e+04,36137.000000,3.804300e+04,3.833700e+04,3.791900e+04,3.833500e+04,3.877700e+04,3.892200e+04,3.878200e+04,3.900100e+04,3.903400e+04,3.937500e+04,32492.000000,24976.000000,3.996400e+04
mode,6.588100e+02,2.000000e+03,3140.613849,9.689000e+03,1.858300e+04,2.188500e+04,2.856180e+04,1.530900e+04,5.120004e+05,1.026612e+06,6.205000e+05,1.448855e+05,1.000000e+00,0.500000,0.000000,4.206858e+06


Africa / Middle East


/opt/mamba/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


,market_cap,employees,revenue,scope_1,scope_2,scope_3,waste_production,waste_recycling,water_consumption,water_withdrawal,energy_consumption,hours_of_training,gender_pay_gap,independent_board_members_percentage,legal_costs_paid_for_controversies,ceo_compensation
moyenne,9.236338e+03,11784.480803,3849.869111,2.016690e+06,2.779231e+05,6.945915e+06,6.490188e+06,5.238273e+05,1.582909e+08,9.753648e+07,9.485931e+06,6.036684e+05,18.763000,0.625773,28.673304,7.776806e+06
min,2.169200e+02,1.000000,-162.340000,0.000000e+00,1.305000e+00,5.140000e-01,8.000000e-02,1.000000e+00,2.624000e+03,0.000000e+00,4.493333e+00,2.500000e+00,-6.000000,0.001500,0.000000,3.363800e+05
max,2.105957e+06,453000.000000,401149.160000,2.457700e+08,4.295000e+07,1.876925e+09,3.146550e+08,2.052540e+07,2.914766e+10,1.221952e+10,6.715740e+08,1.420000e+07,36.000000,1.000000,3728.660000,7.168574e+07
std,9.722486e+04,29913.217732,18486.726780,1.540549e+07,2.127480e+06,9.425033e+07,3.673485e+07,2.393397e+06,2.081794e+09,8.364442e+08,5.863856e+07,1.687317e+06,11.595173,0.258741,190.109756,1.016347e+07
moyennelog,7.668069e+00,8.179699,6.680018,9.240681e+00,9.458262e+00,9.988298e+00,9.956917e+00,8.761799e+00,1.334443e+01,1.413453e+01,1.253849e+01,1.004448e+01,2.764803,0.471744,1.054921,1.524405e+01
minlog,5.384128e+00,0.693147,-0.430783,0.000000e+00,8.350807e-01,4.147552e-01,7.696104e-02,6.931472e-01,7.872836e+00,0.000000e+00,1.703535e+00,1.252763e+00,0.000000,0.001499,0.000000,1.272600e+01
maxlog,1.456028e+01,13.023650,12.902091,1.931991e+01,1.757555e+01,2.135290e+01,1.956699e+01,1.683717e+01,2.409564e+01,2.322630e+01,2.032513e+01,1.646875e+01,3.610918,0.693147,8.224072,1.808780e+01
stdlog,1.171320e+00,1.641964,1.740361,3.552264e+00,2.832688e+00,3.678273e+00,3.576483e+00,3.093397e+00,2.442597e+00,2.688897e+00,2.998853e+00,4.116080e+00,1.008218,0.174267,1.662804,1.140418e+00
nb_values,8.540000e+02,1039.000000,1379.000000,4.380000e+02,4.400000e+02,3.930000e+02,2.050000e+02,1.310000e+02,3.710000e+02,3.510000e+02,2.450000e+02,1.680000e+02,18.000000,109.000000,206.000000,1.140000e+02
mode,2.105957e+06,50000.000000,15.890000,4.000000e+00,1.000000e+02,2.145330e+03,3.611100e+04,1.000000e+00,4.983930e+05,4.983930e+05,1.495640e+08,7.100000e+00,36.000000,0.500000,0.000000,6.100000e+06


...

Latin America and Caribbean


,market_cap,employees,revenue,scope_1,scope_2,scope_3,waste_production,waste_recycling,water_consumption,water_withdrawal,energy_consumption,hours_of_training,gender_pay_gap,independent_board_members_percentage,legal_costs_paid_for_controversies,ceo_compensation
moyenne,3228.0812578616355,10360.803462321792,2735.936534090909,601566.8903023953,283613.23303030303,2255008.7203322784,4320070.32982906,342440.6094086022,7459759.462891966,28955250.966593403,82904772.55619052,680781.9437395282,19.256000000000004,0.6042483294671556,16.47447852760736,5186694.072380953
min,64.41,2.0,-8257.94,1.098,8.419,1.244,15.3,2.0,0.0,1275.5,1117.516666756068,3.6,0.0,0.003125,0.0,417910.66
max,57648.43,280747.0,189115.09,76175428.0,13550000.0,188700000.0,198420000.0,24695100.0,267130000.0,1160462251.57,11493333334.0,8500000.0,43.40000000000001,1.0,1083.92,38967196.0
std,6612.3209809838645,24178.25433751663,8849.089995926424,4456354.101340609,1237768.7845170635,14106673.827233825,25198426.14701985,2543951.0346864774,27182901.190089807,129642953.34527174,929105287.5202868,1353116.6241434484,12.110397626282413,0.24522085364941776,81.28043286204472,6802177.512598652
moyennelog,7.288434455430368,7.930233607919124,6.287238027430539,9.025873940339139,9.387600858726465,9.905430609688832,10.000443111172153,9.044951866994706,13.003485432199694,13.874965312982969,13.233040157504359,11.181252228507837,2.7029444182732187,0.46008989457266525,0.7911946929440047,14.822972414289755
minlog,4.180675151973121,1.0986122886681096,0.18232155679395465,0.7409845099741055,2.24272892584068,0.8082599876604498,2.7911651078127164,1.0986122886681096,0.0,7.151877236681538,7.01975868180805,1.5260563034950492,0.0,0.003120127336243634,0.0,12.94302534945859
maxlog,10.962135638397674,12.545212748799713,12.150116338282254,18.148549514589142,16.42189717909072,19.055669015638195,19.105896559225844,17.022115441845237,19.403245993000635,20.872084256282026,23.165032994218805,15.955576839107596,3.7932394694381792,0.6931471805599453,6.989261530537275,17.478230747604584
stdlog,1.1264064280982096,1.7231767042215809,1.7847543916905704,3.309625229130875,2.9827170817680617,3.418436593811311,3.1372404716182793,2.8551222366192452,2.931339543691008,2.67806734353505,2.4341400894279936,3.5158572278022433,0.9558624844195258,0.16184292672142842,1.4784490324650628,1.1144314410079224
nb_values,750,907,1230,333,330,311,130,93,298,274,153,107,16,87,145,64
mode,559.0,500.0,86.39,22.0,211.0,31.264,163389.325,192.0,0.0,48312102.0,7951262.0,1525216.0,nan,0.5,0.0,nan


United States and Canada


/opt/mamba/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/mamba/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/mamba/lib/python3.11/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,market_cap,employees,revenue,scope_1,scope_2,scope_3,waste_production,waste_recycling,water_consumption,water_withdrawal,energy_consumption,hours_of_training,gender_pay_gap,independent_board_members_percentage,legal_costs_paid_for_controversies,ceo_compensation
moyenne,1.373177e+04,2.237694e+04,6205.847871,2.615906e+06,8.490731e+05,1.674783e+07,2.561851e+07,3.075844e+05,5.824868e+07,2.293765e+08,1.426453e+07,1.316559e+06,9.222981,0.614905,63.777145,1.522916e+07
min,1.458700e+02,1.000000e+00,-2624.020000,0.000000e+00,0.000000e+00,0.000000e+00,7.000000e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,8.000000e-01,-8.690000,0.000036,0.000000,1.563160e+05
max,2.251352e+06,2.300000e+06,452573.520000,5.538870e+08,4.059380e+08,1.679000e+10,2.797400e+10,4.800000e+07,5.794390e+10,9.156512e+10,7.204780e+08,1.482950e+08,44.400000,1.000000,14518.780000,4.220000e+09
std,7.007444e+04,6.710277e+04,20772.496000,2.074022e+07,1.258249e+07,3.832663e+08,7.319472e+08,2.274168e+06,1.330561e+09,2.935619e+09,5.069344e+07,5.099247e+06,11.557736,0.272089,469.813908,1.481164e+08
moyennelog,7.942337e+00,8.500829e+00,6.968086,9.289543e+00,9.674648e+00,1.035623e+01,1.031932e+01,9.631298e+00,1.363540e+01,1.434096e+01,1.357812e+01,1.054031e+01,-inf,0.463578,1.189096,1.544801e+01
minlog,4.989548e+00,6.931472e-01,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,6.765865e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.877867e-01,-inf,0.000036,0.000000,1.195964e+01
maxlog,1.462704e+01,1.464842e+01,13.022708,2.013247e+01,1.982171e+01,2.354405e+01,2.405454e+01,1.768671e+01,2.478274e+01,2.524032e+01,2.039543e+01,1.881471e+01,3.815512,0.693147,9.583267,2.216310e+01
stdlog,1.528679e+00,1.815274e+00,1.923396,3.715705e+00,3.112463e+00,3.885394e+00,3.049830e+00,2.753978e+00,2.776831e+00,2.800315e+00,2.853187e+00,4.414742e+00,NaN,0.182139,1.860615,1.246977e+00
nb_values,4.321000e+03,4.993000e+03,7725.000000,2.745000e+03,2.786000e+03,2.486000e+03,1.413000e+03,1.019000e+03,2.320000e+03,2.158000e+03,1.658000e+03,1.050000e+03,72.000000,309.000000,1006.000000,8.010000e+02
mode,6.069600e+02,2.100000e+04,0.130000,1.000000e+00,8.000000e+03,1.700000e+01,1.700000e+03,5.850000e+02,0.000000e+00,0.000000e+00,1.881410e+07,2.800000e+01,0.000000,1.000000,0.000000,3.531880e+05
